This notebook will develop a function to read wideband frequency response output from Echoview and adjust the frequency bins to resample at a given frequency range points.

In [1]:
import csv
import numpy as np
import pandas as pd
import scipy as sc
import matplotlib.pyplot as plt
from scipy.interpolate import UnivariateSpline
import seaborn as sns
import array as arr

import inverse_method_funs as inv
%matplotlib inline

In [2]:
sns.set_theme(style="darkgrid")
sns.color_palette("viridis", as_cmap=True)

# Write function to read file

In [ ]:
def read_scatteringmodelsimulations(fname,nsim, ve=False):
    """"Opens a file created by R for ZooScatR containing all the information of the model runs. 
    Extract the frequencies, species and cross-sectional backscatter data.
    
    Parameters:
    fname: filename with path of EV export from wideband frequency response plot - graph- export
    nsim: number of simulations
    ve: whether to use model solutions available from viscous elastic model, if species exist in 
        fname, the sigbsmean is replaced with viscous elastic mean results, else a column is added. Default False. 

    
    Returns:
    spec: array of name of species
    freqs: array of frequencies
    sigma_bs_mean: array of mean cross-sectional backscatter simulation values 
    """
    simulations = pd.read_feather(fname)
    # Add a row for sigma_bs calculated from TS
    simulations['sigbs'] = 10**(simulations.TS/10)
    
    # Set dimensions and their values
    freqs = simulations['freq'].unique()
    specs = simulations['spec'].unique()
    
    # Allocate space
    sigma_bs_mean = np.zeros([len(freqs), len(specs)])
    ci_boot = np.zeros([2, len(freqs), len(specs)])
    #Sort values in sibgs column by species and frequencies
    simulations_group = simulations.groupby(["spec","freq"]).agg({'sigbs':lambda x: arr.array('d', x)})
    
    #for mean output
    simulations_mean = simulations.groupby(["spec","freq"]).agg({'sigbs':'mean'})
    mean_bs = simulations_mean.reset_index('freq')

    #Manually organize into an array
    for ind_spec in range(0, len(specs)):
        spec_name = specs[ind_spec]
        sim_spec = np.asarray(simulations_group.sigbs[spec_name])   
        sigma_bs_mean[:,ind_spec] = mean_bs.loc[spec_name].to_numpy()[:,1]
        
        sigma_bs_mean[:,ind_spec], ci_boot[:,:,ind_spec] = inv.bootstrap_interval(simulations, spec=spec_name)
        
        
    if ve==True:
        cod_scat = pd.read_csv('../ViscousElasticModel/ve_results/ve_simulations_cod.txt', header=None, delimiter=' ', names=['frequency', 'TS'], skiprows=1)
        cod_scat['sigbs'] = 10**(cod_scat['TS']/10)
        cod_sigbs_ve, cod_ci_ve = inv.bootstrap_interval(cod_scat, spec=False)
        freqs_cod = cod_scat['frequency'].unique()/1000
        
        #resample frequency and append or replace
        f = UnivariateSpline(freqs_cod,cod_sigbs_ve, k=3, s=2)
        f_ci1 = UnivariateSpline(freqs_cod,cod_ci_ve[0,:], k=3, s=2)
        f_ci2 = UnivariateSpline(freqs_cod,cod_ci_ve[1,:], k=3, s=2)
        # Check if this species is already accounted for in lengths and shape of preallocated result matrices.
        # If yes, put the data where it belongs. If not append
        if (specs=='FishLarvae').any():
            cod_index=np.where(specs=='FishLarvae')
            sigma_bs_mean[:,cod_index[0][0]] = f(freqs)
            ci_boot[0,:,cod_index[0][0]] = f_ci1(freqs)
            ci_boot[1,:,cod_index[0][0]] = f_ci2(freqs)
        else:
            sigma_bs_mean = np.vstack((sigma_bs_mean.T,[f(freqs)]))
            ci1_boot = ci_boot[0,:,:]
            ci2_boot = ci_boot[1,:,:]
            print(np.shape(ci_boot))
            ci1_boot = np.vstack((ci1_boot.T,[f_ci1(freqs)]))
            ci2_boot = np.vstack((ci2_boot.T,[f_ci2(freqs)]))
            sigma_bs_mean = sigma_bs_mean.T
            ci_boot = np.array([ci1_boot.T, ci2_boot.T])
            specs = np.append(specs,'FishLarvae')
        
        
        
        lima_scat = pd.read_csv('../ViscousElasticModel/ve_results/ve_simulations_limacina.txt', header=None, delimiter=' ', names=['frequency', 'TS'], skiprows=1)
        lima_scat['sigbs'] = 10**(lima_scat['TS']/10)
        lima_sigbs_ve, lima_ci_ve = inv.bootstrap_interval(lima_scat, spec=False)
        freqs_lima = lima_scat['frequency'].unique()/1000
        #resample frequency and append or replace   
        f = UnivariateSpline(freqs_lima,lima_sigbs_ve, k=5)  
        f_ci1 = UnivariateSpline(freqs_lima,lima_ci_ve[0,:], k=3, s=2)
        f_ci2 = UnivariateSpline(freqs_lima,lima_ci_ve[1,:], k=3, s=2)

        # Check if this species is already accounted for in lengths and shape of preallocated result matrices.
        # If yes, put the data where it belongs. If not append!
        if (specs=='Pteropod').any():
            lima_index = np.where(specs=='Pteropod')
            sigma_bs_mean[:,lima_index[0][0]] = f(freqs)
            ci_boot[0,:,lima_index[0][0]] = f_ci1(freqs)
            ci_boot[1,:,lima_index[0][0]] = f_ci2(freqs)
        else:                    
            sigma_bs_mean = np.vstack((sigma_bs_mean.T,[f(freqs)]))
            print(np.shape(ci_boot))
            ci1_boot = ci_boot[0,:,:]
            ci2_boot = ci_boot[1,:,:]

            ci1_boot = np.vstack((ci1_boot.T,[f_ci1(freqs)]))
            ci2_boot = np.vstack((ci2_boot.T,[f_ci2(freqs)]))
            sigma_bs_mean = sigma_bs_mean.T
            ci_boot = np.array([ci1_boot.T, ci2_boot.T])
            specs = np.append(specs,'Pteropod')                                         
    
    
    return specs, freqs, sigma_bs_mean,ci_boot

# Run function

In [ ]:
fname = '../ZooScatStuff/my_simplified_data.feather'
specs, freqs, sig_mean, ci_boot = read_scatteringmodelsimulations(fname, nsim=1000, ve=True)

In [4]:
fname = '../ZooScatStuff/my_simplified_data.feather'
specs, freqs, sig_mean_inv, ci_boot = inv.read_scatteringmodelsimulations(fname, nsim=1000, ve=True)

(2, 801, 3)
(2, 801, 4)


In [ ]:
lima_scat = pd.read_csv('../ViscousElasticModel/ve_results/ve_simulations_limacina.txt', header=None, delimiter=' ', names=['frequency', 'TS'], skiprows=1)
lima_scat

In [ ]:
ci_boot

# Plot

In [ ]:
fig=plt.figure(figsize=(10, 10),facecolor='w', edgecolor='k')
plt.plot(freqs, sig_mean)
plt.title('Mean from DWBA and VE simulations')
plt.legend(specs)
plt.xlabel('Frequency (kHz)')
plt.ylabel('$s_{v}$ [$m^{-1}$]');

In [ ]:
cod_scat = pd.read_csv('../ViscousElasticModel/ve_results/ve_simulations_cod.txt', header=None, delimiter=' ', names=['frequency', 'TS'], skiprows=1)
cod_sigbs_ve = 10**(cod_scat['TS']/10)
freqs_cod = cod_scat['frequency'].unique()/1000

In [ ]:
f = UnivariateSpline(freqs_cod,cod_sigbs_ve, k=3, s=2)

In [ ]:
np.shape(cod_sigbs_ve)

In [ ]:
for ind_spec in range(0, len(specs)):
    plt.plot(freqs, sig_mean[:,ind_spec])
    plt.fill_between(freqs, ci_boot[0,:,ind_spec], ci_boot[1,:,ind_spec], alpha=0.5)
plt.legend(specs)

In [ ]:
ind_spec=4
plt.plot(freqs, 10*np.log10(sig_mean[:,ind_spec]),'r',label='pteropod ci',linewidth=12)
plt.plot(freqs,10*np.log10(sig_mean_inv[:,ind_spec]),'k',label='pteropod')
ind_spec=3
plt.plot(freqs, 10*np.log10(sig_mean[:,ind_spec]),'g',label='fish ci', linewidth=16)
plt.plot(freqs,10*np.log10(sig_mean_inv[:,ind_spec]),'b',label='fish')
    #plt.fill_between(freqs, 10*np.log10(ci_boot[0,:,ind_spec]), 10*np.log10(ci_boot[1,:,ind_spec]), alpha=0.5)
plt.legend()


In [ ]:
ci_boot[1,:,:]